### Importing libraries

In [1]:
from ImgOps import *
import copy as cp
from sklearn.cluster import DBSCAN

### Imreading images

In [2]:
query_img = cv.imread('query_4.jpg')
query_img_gray = cv.cvtColor(query_img, cv.COLOR_BGR2GRAY)
imshow(query_img)

train_img = cv.imread('train_4.jpg')
train_img_gray = cv.cvtColor(train_img, cv.COLOR_BGR2GRAY)
imshow(train_img)

### Adding keypoints via SIFT

In [3]:
# Initiate SIFT detector
sift = cv.SIFT_create()

# find the keypoints and descriptors with SIFT
query_kps, query_des = sift.detectAndCompute(query_img_gray, None)
train_kps, train_des = sift.detectAndCompute(train_img_gray, None)

### Keypoints matching via FLANN

In [4]:
index_params = dict(algorithm=1, trees=5)
search_params = dict(checks=50)
flann = cv.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(query_des, train_des, k=2)
matchesMask = [[0, 0] for _ in range(len(matches))]

good_matches_query, good_matches_train = [], []
for i, (m, n) in enumerate(matches):
    if m.distance < .5 * n.distance:
        matchesMask[i] = [1, 0]
        good_matches_query.append(m)
        good_matches_train.append(n)
good_matches_query = np.asarray(good_matches_query)
good_matches_train = np.asarray(good_matches_train)

draw_params = dict(matchColor=(0, 255, 0),
                   matchesMask=matchesMask,
                   flags=cv.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
matched_img = cv.drawMatchesKnn(query_img, query_kps, train_img, train_kps, matches, None, **draw_params)

imshow(matched_img)

### Clustering via DBSCAN

In [5]:
# Dictionary(ptQuery_ptTrain) = { (x_q, y_q): (x_t), (y_t) }
ptQuery_ptTrain = {}

for DMatch_q, DMatch_t in zip(good_matches_query, good_matches_train):
    pt_q = query_kps[DMatch_q.queryIdx].pt
    pt_t = train_kps[DMatch_t.trainIdx].pt
    ptQuery_ptTrain[pt_q] = pt_t


# Clustering
clusterized = DBSCAN(eps=50, min_samples=5).fit_predict(list(ptQuery_ptTrain.keys()))

# Dictionary(cluster_pts_q) = { cluster_name: [(x_q, y_q), ...] }
cluster_pts_q = {}
for gp, pt in zip(clusterized, list(ptQuery_ptTrain.keys())):
    if gp == -1: continue
    else:
        if gp not in cluster_pts_q:
            cluster_pts_q[gp] = [pt]
        else:
            cluster_pts_q[gp].append(pt)

cluster_pts_t = cp.deepcopy(cluster_pts_q)
for cluster in cluster_pts_t:
    for i, pt in enumerate(cluster_pts_t[cluster]):
        cluster_pts_t[cluster][i] = ptQuery_ptTrain[pt]

### Clusters' visualization

In [6]:
for pts in cluster_pts_q.values():
    for x, y in pts:
        clust_img = cv.circle(query_img, (int(x), int(y)), radius=3, color=(255, 0, 255), thickness=-1)
    imshow(clust_img)
    clust_img = None

In [7]:
ptQuery_ptTrain

{(151.68614196777344, 270.49859619140625): (36.52845001220703,
  27.28488540649414),
 (152.91102600097656, 267.06561279296875): (66.34867095947266,
  36.381103515625),
 (160.05548095703125, 270.32208251953125): (112.34420013427734,
  59.89463424682617),
 (286.5384826660156, 262.591796875): (100.78785705566406, 27.849998474121094),
 (288.956787109375, 307.5103759765625): (146.15272521972656,
  33.75363540649414),
 (289.62890625, 282.8891296386719): (53.123756408691406, 26.2194766998291),
 (292.1750793457031, 262.64990234375): (162.92996215820312,
  29.234397888183594),
 (294.7551574707031, 294.5662536621094): (36.52845001220703,
  27.28488540649414),
 (296.6008605957031, 287.8304138183594): (66.34867095947266, 36.381103515625),
 (298.2112121582031, 269.9421081542969): (87.09210968017578, 70.8189926147461),
 (298.3581237792969, 262.30303955078125): (36.52845001220703,
  27.28488540649414),
 (299.8431701660156, 294.4839782714844): (10.965537071228027,
  9.601112365722656),
 (303.279693603

In [8]:
cluster_pts_q

{0: [(286.5384826660156, 262.591796875),
  (288.956787109375, 307.5103759765625),
  (289.62890625, 282.8891296386719),
  (292.1750793457031, 262.64990234375),
  (294.7551574707031, 294.5662536621094),
  (296.6008605957031, 287.8304138183594),
  (298.2112121582031, 269.9421081542969),
  (298.3581237792969, 262.30303955078125),
  (299.8431701660156, 294.4839782714844),
  (303.2796936035156, 266.1802978515625),
  (308.72174072265625, 294.2988586425781),
  (310.0554504394531, 266.3790283203125),
  (320.2637939453125, 275.2845153808594),
  (339.0787353515625, 264.3700256347656)],
 1: [(575.6551513671875, 140.9866943359375),
  (580.515625, 190.17245483398438),
  (586.98291015625, 172.07022094726562),
  (596.8635864257812, 163.81593322753906),
  (598.7796020507812, 134.25709533691406),
  (605.5413208007812, 103.8653335571289)],
 2: [(655.5125122070312, 64.98111724853516),
  (655.6220703125, 57.96885299682617),
  (661.877685546875, 81.86820220947266),
  (662.9232788085938, 53.468570709228516),

In [9]:
cluster_pts_t

{0: [(100.78785705566406, 27.849998474121094),
  (146.15272521972656, 33.75363540649414),
  (53.123756408691406, 26.2194766998291),
  (162.92996215820312, 29.234397888183594),
  (36.52845001220703, 27.28488540649414),
  (66.34867095947266, 36.381103515625),
  (87.09210968017578, 70.8189926147461),
  (36.52845001220703, 27.28488540649414),
  (10.965537071228027, 9.601112365722656),
  (49.264671325683594, 95.2537841796875),
  (2.9281296730041504, 5.142671585083008),
  (84.62295532226562, 36.58852005004883),
  (25.169227600097656, 59.816650390625),
  (36.52845001220703, 27.28488540649414)],
 1: [(44.7270393371582, 47.00611877441406),
  (27.46978759765625, 90.25101470947266),
  (53.41572189331055, 21.440515518188477),
  (66.34867095947266, 36.381103515625),
  (23.975017547607422, 37.3960075378418),
  (76.64567565917969, 36.57674026489258)],
 2: [(6.930593490600586, 146.24130249023438),
  (6.841889381408691, 139.73887634277344),
  (9.529679298400879, 27.013498306274414),
  (112.344200134277

In [10]:
for cluster in cluster_pts_q:
    print(f'Cluster #{cluster}:\nSize of query cluster = {len(cluster_pts_q[cluster])}\nSize of train cluster = {len(cluster_pts_t[cluster])}', end='\n\n\n')

Cluster #0:
Size of query cluster = 14
Size of train cluster = 14


Cluster #1:
Size of query cluster = 6
Size of train cluster = 6


Cluster #2:
Size of query cluster = 83
Size of train cluster = 83




### Feature Homography

In [11]:
for cluster in cluster_pts_q:
    src = np.float32(cluster_pts_t[cluster]).reshape(-1, 1, 2)
    dst = np.float32(cluster_pts_q[cluster]).reshape(-1, 1, 2)
    
    M, _ = cv.findHomography(src, dst, cv.RANSAC, 5.)
    h, w = train_img_gray.shape
    pts = np.float32([[0, 0], [0, h], [w, h], [w, 0]]).reshape(-1, 1, 2)
    dst = cv.perspectiveTransform(pts, M)
    res_img = cv.polylines(query_img, [np.int32(dst)], True, 255, 3, cv.LINE_AA)

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-9d_dfo3_\opencv\modules\core\src\matmul.dispatch.cpp:531: error: (-215:Assertion failed) scn + 1 == m.cols in function 'cv::perspectiveTransform'


In [ ]:
imshow(res_img)